# RL tréning a Cartpole egyensúlyozására

Ez a jegyzetfüzet az [AI for Beginners Curriculum](http://aka.ms/ai-beginners) része. Az ötletet az [official PyTorch tutorial](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) és [ez a Cartpole PyTorch implementáció](https://github.com/yc930401/Actor-Critic-pytorch) adta.

Ebben a példában RL-t fogunk használni, hogy egy modellt tanítsunk arra, hogy egy rudat egyensúlyozzon egy kocsin, amely vízszintes skálán balra és jobbra mozoghat. Az [OpenAI Gym](https://www.gymlibrary.ml/) környezetet fogjuk használni a rúd szimulálására.

> **Note**: A lecke kódját futtathatod helyben (például a Visual Studio Code-ban), ebben az esetben a szimuláció egy új ablakban nyílik meg. Ha a kódot online futtatod, előfordulhat, hogy néhány módosítást kell végezned a kódon, ahogy azt [itt](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7) leírták.

Először győződjünk meg arról, hogy a Gym telepítve van:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Most készítsük el a CartPole környezetet, és nézzük meg, hogyan működhetünk benne. Egy környezetnek a következő tulajdonságai vannak:

* **Akciótér**: az elérhető akciók halmaza, amelyeket a szimuláció minden lépésében végrehajthatunk  
* **Megfigyelési tér**: az a tér, amelyben a megfigyeléseinket végezhetjük


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Nézzük meg, hogyan működik a szimuláció. Az alábbi ciklus futtatja a szimulációt, amíg az `env.step` nem adja vissza a `done` befejezési jelzőt. Véletlenszerűen választunk akciókat az `env.action_space.sample()` segítségével, ami azt jelenti, hogy a kísérlet valószínűleg nagyon gyorsan kudarcot vall (a CartPole környezet akkor ér véget, ha a CartPole sebessége, pozíciója vagy szöge bizonyos határokon kívül esik).

> A szimuláció egy új ablakban fog megnyílni. Többször is futtathatod a kódot, és megnézheted, hogyan viselkedik.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Megfigyelheted, hogy a megfigyelések 4 számot tartalmaznak. Ezek a következők:
- A kocsi pozíciója
- A kocsi sebessége
- A rúd szöge
- A rúd forgási sebessége

`rew` az a jutalom, amit minden lépésnél kapunk. Láthatod, hogy a CartPole környezetben minden szimulációs lépésért 1 pontot kapsz jutalomként, és a cél az összjutalom maximalizálása, azaz az idő, ameddig a CartPole egyensúlyban tud maradni anélkül, hogy leesne.

A megerősítéses tanulás során a célunk egy **politika** $\pi$ betanítása, amely minden állapot $s$ esetén megmondja, hogy melyik akciót $a$ kell végrehajtanunk, tehát lényegében $a = \pi(s)$.

Ha valószínűségi megoldást szeretnél, a politikát úgy is elképzelheted, hogy minden akcióhoz egy valószínűséget ad vissza, azaz $\pi(a|s)$ azt jelentené, hogy mekkora a valószínűsége annak, hogy az $s$ állapotban az $a$ akciót kell végrehajtanunk.

## Policy Gradient Módszer

A legegyszerűbb RL algoritmusban, amit **Policy Gradient**-nek nevezünk, egy neurális hálót fogunk betanítani arra, hogy megjósolja a következő akciót.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

A hálózatot úgy fogjuk betanítani, hogy sok kísérletet futtatunk, és minden futtatás után frissítjük a hálózatunkat. Határozzunk meg egy függvényt, amely lefuttatja a kísérletet, és visszaadja az eredményeket (az úgynevezett **nyomvonalat**) - az összes állapotot, akciót (és azok ajánlott valószínűségeit), valamint a jutalmakat:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Futtathatsz egy epizódot egy betanítatlan hálózattal, és megfigyelheted, hogy a teljes jutalom (más néven az epizód hossza) nagyon alacsony:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

A policy gradient algoritmus egyik trükkös aspektusa a **diszkontált jutalmak** használata. Az ötlet az, hogy kiszámítjuk a teljes jutalmak vektorát a játék minden lépésénél, és közben a korai jutalmakat diszkontáljuk egy $gamma$ együtthatóval. Az így kapott vektort normalizáljuk is, mivel súlyként fogjuk használni, hogy befolyásoljuk a tanításunkat:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Most kezdjük el az igazi gyakorlást! 300 epizódot fogunk futtatni, és minden epizódban a következőket tesszük:

1. Futtatjuk a kísérletet, és összegyűjtjük a nyomvonalat.
2. Kiszámítjuk a különbséget (`gradients`) a végrehajtott akciók és az előrejelzett valószínűségek között. Minél kisebb a különbség, annál biztosabbak lehetünk abban, hogy a helyes akciót választottuk.
3. Kiszámítjuk a diszkontált jutalmakat, és megszorozzuk a gradiens értékeket a diszkontált jutalmakkal - ez biztosítja, hogy a magasabb jutalommal járó lépések nagyobb hatással legyenek a végső eredményre, mint az alacsonyabb jutalmúak.
4. A neurális hálózatunk számára elvárt célakciók részben a futtatás során előrejelzett valószínűségekből, részben pedig a kiszámított gradiens értékekből származnak. Az `alpha` paraméter segítségével határozzuk meg, hogy milyen mértékben vesszük figyelembe a gradiens értékeket és a jutalmakat - ezt nevezzük a megerősítéses algoritmus *tanulási rátájának*.
5. Végül betanítjuk a hálózatunkat az állapotok és az elvárt akciók alapján, majd megismételjük a folyamatot.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Most nézzük meg az epizódot rendereléssel, hogy lássuk az eredményt:


In [ ]:
_ = run_episode(render=True)

Remélhetőleg most már látható, hogy a rúd elég jól tud egyensúlyozni!

## Actor-Critic Modell

Az Actor-Critic modell a policy gradientek továbbfejlesztése, amelyben egy neurális hálót építünk, hogy megtanulja mind a policy-t, mind a becsült jutalmakat. A hálózatnak két kimenete lesz (vagy tekinthetjük két különálló hálózatnak is):
* **Actor** ajánlja a végrehajtandó akciót azáltal, hogy megadja az állapot valószínűségi eloszlását, hasonlóan a policy gradient modellhez.
* **Critic** megbecsüli, hogy ezekből az akciókból milyen jutalom származhat. Visszaadja a jövőbeli összes becsült jutalmat az adott állapotban.

Határozzunk meg egy ilyen modellt:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Szükségünk lenne arra, hogy kissé módosítsuk a `discounted_rewards` és a `run_episode` függvényeinket:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Most elindítjuk a fő tanítási ciklust. A hálózat kézi tanítási folyamatát fogjuk használni, megfelelő veszteségfüggvények kiszámításával és a hálózati paraméterek frissítésével:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Összefoglaló

Ebben a bemutatóban két megerősítéses tanulási (RL) algoritmust láttunk: az egyszerű policy gradientet és a kifinomultabb actor-critic módszert. Látható, hogy ezek az algoritmusok az állapot, cselekvés és jutalom absztrakt fogalmaival dolgoznak – így nagyon különböző környezetekben is alkalmazhatók.

A megerősítéses tanulás lehetővé teszi számunkra, hogy megtanuljuk a legjobb stratégiát a probléma megoldására, pusztán a végső jutalom megfigyelésével. Az a tény, hogy nincs szükség címkézett adathalmazokra, lehetővé teszi számunkra, hogy sokszor megismételjük a szimulációkat, hogy optimalizáljuk modelljeinket. Ugyanakkor az RL területén még mindig számos kihívás van, amelyeket megismerhetsz, ha úgy döntesz, hogy mélyebben elmerülsz ebben az izgalmas AI területben.



---

**Felelősségkizárás**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével készült. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt a professzionális, emberi fordítás igénybevétele. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
